Label Studio Video FPS is 25.04

In [ ]:
import cv2
import json
import os
import numpy as np

# Load the annotations JSON file
json_path = r'video1.json'
with open(json_path, 'r') as f:
    annotations = json.load(f)

# Output video directory
output_dir = 'output_videos'
os.makedirs(output_dir, exist_ok=True)

# Define the FPS of the video and Label Studio
video_fps = 59.19  # Actual video FPS
label_studio_fps = 25.04  # Label Studio FPS
scaling_factor = video_fps / label_studio_fps  # Scaling factor

# Function to check if a bounding box should be drawn in the current frame
def get_boxes_for_frame(sequence, frame_number):
    for i in range(len(sequence) - 1):
        start_box = sequence[i]
        end_box = sequence[i + 1]

        # Scale Label Studio frames to match video FPS
        start_frame = int(start_box['frame'] * scaling_factor)
        end_frame = int(end_box['frame'] * scaling_factor)
        
        # Check if the current frame falls between the start_frame and end_frame
        if start_frame <= frame_number < end_frame:
            if start_box['enabled']:
                return {
                    'x': start_box['x'],
                    'y': start_box['y'],
                    'width': start_box['width'],
                    'height': start_box['height'],
                    'labels': start_box.get('labels', [])
                }
    # If no enabled box is found, return None
    return None

# Function to draw bounding boxes on a frame
def draw_boxes(frame, boxes):
    for box in boxes:
        if box:
            # Extract box coordinates
            x = int(box['x'] * frame.shape[1] / 100)  # Convert from percentage to pixels
            y = int(box['y'] * frame.shape[0] / 100)
            width = int(box['width'] * frame.shape[1] / 100)
            height = int(box['height'] * frame.shape[0] / 100)

            # Draw rectangle
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)

            # Draw the label if available
            if 'labels' in box and box['labels']:
                label = ', '.join(box['labels'])
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame

# Process each video in the annotations
for annotation in annotations:
    video_path = annotation['video']  # Update this to your local path if needed
    video_filename = os.path.basename(video_path)

    # Load video
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create VideoWriter to save the new video with bounding boxes
    output_video_path = os.path.join(output_dir, f'output_{video_filename}')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Process frame by frame
    frame_number = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Collect all boxes for this frame from all classes
        frame_boxes = []
        for box_annotation in annotation['box']:
            sequence = box_annotation['sequence']
            box_for_frame = get_boxes_for_frame(sequence, frame_number)
            if box_for_frame:
                # Attach labels from the box annotation
                box_for_frame['labels'] = box_annotation.get('labels', [])
                frame_boxes.append(box_for_frame)

        # Draw bounding boxes for the current frame
        frame = draw_boxes(frame, frame_boxes)

        # Write the frame with the bounding boxes
        out.write(frame)

        frame_number += 1
        if frame_number >= total_frames:
            break

    cap.release()
    out.release()

print("Processing complete!")
